<a href="https://colab.research.google.com/github/gackouhamady/FinalProject_ML_NeuralNetworks_ENI-ABT_2023/blob/main/Deep_Learning_with_Keras_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Sommaire :**


**Contexte**

A. **Construction d'un modèle de référence**


B.   **Normalisation des données**

C.   **Incrementation du nombre d'époques**


D.   **Augmentation du nombre de couches cachés**

C.   **Conclusion**









#  **Contexte**
Dans ce projet, nous construirons un modèle de régression à l'aide de la bibliothèque d'apprentissage en profondeur Keras, puis nous expérimenterons l'augmentation du nombre d'époques de formation et la modification du nombre de couches cachées et nous verrons comment la modification de ces paramètres affecte les performances du modèle. Pour la commodité, les données peuvent être retrouvées ici :https://cocl.us/concrete_data .  

# 1. Préparer l'environnement

Tout d'abord, procédons à quelques importations et téléchargeons l'ensemble des données.

In [22]:
import pandas as pd
import numpy as np
import keras

from keras.models import Sequential
from keras.layers import Dense
# charger le jeu de données concrete_data
concrete_data = pd.read_csv('https://cocl.us/concrete_data')

# vérifier que les données ont été chargées correctement
concrete_data.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age,Strength
0,540.0,0.0,0.0,162.0,2.5,1040.0,676.0,28,79.99
1,540.0,0.0,0.0,162.0,2.5,1055.0,676.0,28,61.89
2,332.5,142.5,0.0,228.0,0.0,932.0,594.0,270,40.27
3,332.5,142.5,0.0,228.0,0.0,932.0,594.0,365,41.05
4,198.6,132.4,0.0,192.0,0.0,978.4,825.5,360,44.30


# 2. Connaitre le nombre d'elements de notre dataset

In [23]:
concrete_data.shape

(1030, 9)

# 3. Diviser l'ensemble de données en prédicteurs (entrées) et en cibles (sorties)

In [29]:

predictors = concrete_data.drop(columns=['Strength'])
target = concrete_data['Strength']
n_cols = predictors.shape[1]

# A. **Construction d'un modele de reference**
1. Utilisez la bibliothèque Keras pour construire un réseau neuronal avec les caractéristiques suivantes :

- Une couche cachée de 10 nœuds et une fonction d'activation ReLU.
- Utilisez l'optimiseur adam et l'erreur quadratique moyenne comme fonction de perte.


In [25]:
# Créer le réseau neuronal dans une fonction afin de pouvoir l'utiliser plusieurs fois dans les sections suivantes
def regression_model():
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

model = regression_model()

2. Divisez aléatoirement les données en deux ensembles, l'un pour la formation et l'autre pour le test, en réservant 30 % des données au test. Vous pouvez utiliser la fonction d'aide train_test_split de Scikit-learn.

In [28]:
from sklearn.model_selection import train_test_split

predictors_train, predictors_test, target_train, target_test = train_test_split(predictors, target, 
                                                                                test_size=0.3, random_state=71)
# Regardons la forme de l'ensemble des prédicteurs
predictors_train.shape

(721, 8)

# 3. Entraînez le modèle sur les données d'entraînement en utilisant 50 époches.

In [ ]:
model.fit(predictors_train, target_train, epochs=50)

Epoch 1/50
23/23 [==============================] - 1s 2ms/step - loss: 283146.9062
Epoch 2/50
23/23 [==============================] - 0s 2ms/step - loss: 191580.5000
Epoch 3/50
23/23 [==============================] - 0s 2ms/step - loss: 126736.9062
Epoch 4/50
23/23 [==============================] - 0s 2ms/step - loss: 81988.5859
Epoch 5/50
23/23 [==============================] - 0s 2ms/step - loss: 51477.3906
Epoch 6/50
23/23 [==============================] - 0s 2ms/step - loss: 31271.7305
Epoch 7/50
23/23 [==============================] - 0s 2ms/step - loss: 18303.2422
Epoch 8/50
23/23 [==============================] - 0s 2ms/step - loss: 10525.0527
Epoch 9/50
23/23 [==============================] - 0s 2ms/step - loss: 6139.2671
Epoch 10/50
23/23 [==============================] - 0s 2ms/step - loss: 3823.9370
Epoch 11/50
23/23 [==============================] - 0s 2ms/step - loss: 2739.3987
Epoch 12/50
23/23 [==============================] - 0s 2ms/step - loss: 2206.0159
Ep

# 4.  Évaluez le modèle sur les données d'essai et calculez l'erreur quadratique moyenne entre la résistance prédite du béton et la résistance réelle du béton. Vous pouvez utiliser la fonction mean_squared_error de Scikit-learn.

In [30]:
predictions_test = model.predict(predictors_test)
from sklearn.metrics import mean_squared_error
print('Erreur quadratique moyenne sur les données de test est %.3f' % (mean_squared_error(target_test, predictions_test)))

10/10 [==============================] - 0s 2ms/step
Erreur quadratique moyenne sur les données de test est 28053.966


5. Répétez les étapes 2 à 4, 50 fois, c'est-à-dire créez une liste de 50 erreurs quadratiques moyennes.
6. Indiquez la moyenne et l'écart-type des erreurs quadratiques moyennes.

In [33]:
# Créer une fonction qui évalue le modèle afin que nous puissions l'utiliser pour évaluer les modèles créés dans les parties A, B, C et D
# Le paramètre 'create_model_func' est la fonction utilisée pour construire le modèle. Pour la partie A, il s'agit de la fonction
# fonction regression_model définie ci-dessus
def evaluate_model(create_model_func, predictors, targets, epochs=50):
    mean_squared_errors = []
    for i in range(50):
      # créer le modèle. nous  n'étions pas sûr à 100% que cela devait se faire à l'intérieur de la boucle, mais nous  pensons que c'était.
      # l'intention de la question. Sinon, la moyenne et l'écart-type de l'erreur quadratique moyenne ne sont pas très significatifs
        model = create_model_func()
     # 1. diviser les données en un ensemble de formation et un ensemble de test
        predictors_train, predictors_test, target_train, target_test = train_test_split(predictors, target, 
                                                                                    test_size=0.3, random_state=71)
       # 2. 50 époches d'entraînement (suppression de l'enregistrement cette fois)
        model.fit(predictors_train, target_train, epochs=epochs, verbose=0)
       # 3. mesurer le mse et l'ajouter à la liste
        predictions_test = model.predict(predictors_test)
        mse = mean_squared_error(target_test, predictions_test)
        mean_squared_errors.append(mse)
        print('.', end='') # affiche un point pour que nous puissions voir que la fonction est toujours en cours d'exécution
    print(' Done!')
   # renvoie la moyenne et l'écart-type de la liste mse
    return np.mean(mean_squared_errors), np.std(mean_squared_errors)

In [34]:
# Évaluer le modèle et imprimer la moyenne et l'écart-type des erreurs quadratiques moyennes. Notez que nous passons ici 
# la _fonction_modèle_ de régression ici. Elle est utilisée dans la fonction evaluate_model pour créer un nouveau # réseau neuronal à chaque boucle.
# nouveau réseau neuronal dans chaque boucle
mean_mse, std_mse = evaluate_model(regression_model, predictors, target)
# Report the mean and stddev of the mean squared errors
print("Erreurs quadratiques moyennes pour 50 modèles de régression : moyenne = %.3f, std dev = %.3f" %(mean_mse, std_mse))

10/10 [==============================] - 0s 2ms/step
. Done!
Mean squared errors for 50 regression models: mean = 312.928, std dev = 351.551


# **B. Normalisation des donnees**

Répétez la partie A mais utilisez une version normalisée des données. Rappelez qu'une façon de normaliser les données consiste à soustraire la moyenne des prédicteurs individuels et à la diviser par l'écart-type.

In [36]:
# Normaliser les données
predictors_norm = (predictors - predictors.mean()) / predictors.std()
predictors_norm.head()

,Cement,Blast Furnace Slag,Fly Ash,Water,Superplasticizer,Coarse Aggregate,Fine Aggregate,Age
0,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,0.862735,-1.217079,-0.279597
1,2.476712,-0.856472,-0.846733,-0.916319,-0.620147,1.055651,-1.217079,-0.279597
2,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,3.551340
3,0.491187,0.795140,-0.846733,2.174405,-1.038638,-0.526262,-2.239829,5.055221
4,-0.790075,0.678079,-0.846733,0.488555,-1.038638,0.070492,0.647569,4.976069


In [37]:
mean_mse, std_mse = evaluate_model(regression_model, predictors_norm, target)
# Indiquer la moyenne et l'écart-type des erreurs quadratiques moyennes
print("Erreurs quadratiques moyennes pour 50 modèles de régression sur des données normalisées : moyenne = %.3f, std dev = %.3f" % 
      (mean_mse, std_mse))

10/10 [==============================] - 0s 2ms/step
. Done!
Mean squared errors for 50 regression models on normalized data: mean = 348.358, std dev = 133.863


**Comment la moyenne des erreurs quadratiques moyennes se compare-t-elle à celle de l'étape A ?**

La moyenne des erreurs quadratiques moyennes a légèrement baissé (~10%), mais l'écart-type a baissé de manière significative. Cela signifie qu'en utilisant des données normalisées, la performance sur l'ensemble de test dépend beaucoup moins de la façon dont les données ont été divisées (aléatoirement) en un ensemble de données d'entraînement et un ensemble de données de test.

# C. Incrementation du nombre d'époches

Répétez la partie B **mais utilisez cette fois 100 époques pour l'entraînement.**

In [ ]:
mean_mse, std_mse = evaluate_model(regression_model, predictors_norm, target, epochs=100)
# Report the mean and stddev of the mean squared errors
print("Mean squared errors for 50 regression models on normalized data, trained 100 epochs: mean = %.3f, std dev = %.3f" % 
      (mean_mse, std_mse))

.................................................. Done!
Mean squared errors for 50 regression models on normalized data, trained 100 epochs: mean = 149.652, std dev = 10.491


**How does the mean of the mean squared errors compare to that from Step B?**

The mean squared error on the test set more than halved. Clearly, there was plenty of improvement to be gained with additional training. The standard deviation has gone down a lot as well.

# Part D - increate the number of hidden layers

Repeat part B but use a neural network with the following instead:

- Three hidden layers, each of 10 nodes and ReLU activation function.

In [ ]:
def regression_model_D():
    model = Sequential()
    model.add(Dense(10, activation='relu', input_shape=(n_cols,)))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# Evaluate model D
mean_mse, std_mse = evaluate_model(regression_model_D, predictors_norm, target)
# Report the mean and stddev of the mean squared errors
print("Mean squared errors for 50 regression models with 3 hidden layers: mean = %.3f, std dev = %.3f" % 
      (mean_mse, std_mse))

.................................................. Done!
Mean squared errors for 50 regression models with 3 hidden layers: mean = 122.131, std dev = 13.626


**How does the mean of the mean squared errors compare to that from Step B?**

The mean of the mean squared errors is a lot better than in part B (123 vs 321). This shows that the network with multiple hidden layers was significantly better at learning to predict the 'Strength' feature than the single layer network.